# Hello AOL

An EDA of the 2006 AOL User Search Query Dataset

In [62]:
# mount google drive

from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [63]:
directory = '/gdrive/My Drive/aol_data/'
common_file_name = 'user-ct-test-collection-'
file_names = []

for number in range(1, 11):
    if number != 10:
        number = f"0{number}"

    file_name = common_file_name + str(number) + '.txt'
    file_names.append(file_name)

file_paths = [directory + file_name for file_name in file_names]
print(file_paths)

['/gdrive/My Drive/aol_data/user-ct-test-collection-01.txt', '/gdrive/My Drive/aol_data/user-ct-test-collection-02.txt', '/gdrive/My Drive/aol_data/user-ct-test-collection-03.txt', '/gdrive/My Drive/aol_data/user-ct-test-collection-04.txt', '/gdrive/My Drive/aol_data/user-ct-test-collection-05.txt', '/gdrive/My Drive/aol_data/user-ct-test-collection-06.txt', '/gdrive/My Drive/aol_data/user-ct-test-collection-07.txt', '/gdrive/My Drive/aol_data/user-ct-test-collection-08.txt', '/gdrive/My Drive/aol_data/user-ct-test-collection-09.txt', '/gdrive/My Drive/aol_data/user-ct-test-collection-10.txt']


In [64]:
# test connection -> True if valid data

expected = 'AnonID\tQuery\tQueryTime\tItemRank\tClickURL\n'

def test_file_data():
    is_valid = None
    for file_path in file_paths:
        with open(file_path, 'r') as f:
            actual = f.readline()

            if expected != actual:
                 is_valid = False
                 break

    if is_valid == None:
        is_valid = True

    return is_valid

print(f'Data is valid: {test_file_data()}')

Data is valid: True


In [131]:
import pandas as pd
import numpy as np

In [132]:
# each file is like 200MB in size
# this is a POC so I am only going to do the first one
converters = {
    'AnonID': np.uint32
}

df = pd.read_csv(file_paths[0], sep='\t', dtype=object, converters=converters)

<ipython-input-132-c6c9bd1b0aa2>:7: ParserWarning: Both a converter and dtype were specified for column AnonID - only the converter will be used.
  df = pd.read_csv(file_paths[0], sep='\t', dtype=object, converters=converters)


In [133]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3558411 entries, 0 to 3558410
Data columns (total 5 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   AnonID     uint32
 1   Query      object
 2   QueryTime  object
 3   ItemRank   object
 4   ClickURL   object
dtypes: object(4), uint32(1)
memory usage: 122.2+ MB


In [134]:
df.head()

,AnonID,Query,QueryTime,ItemRank,ClickURL
0,142,rentdirect.com,2006-03-01 07:17:12,NaN,NaN
1,142,www.prescriptionfortime.com,2006-03-12 12:31:06,NaN,NaN
2,142,staple.com,2006-03-17 21:19:29,NaN,NaN
3,142,staple.com,2006-03-17 21:19:45,NaN,NaN
4,142,www.newyorklawyersite.com,2006-03-18 08:02:58,NaN,NaN


## Data Cleaning

ItemRank and ClickURL have NaN datatypes. Ideally, we want a number 0 and for ItemRank.

For ClickURL, we will leave NaN values but process the URLs to keep the netloc. We keep NaNs to group by later, I guess.

In [135]:
# by setting 0, take advantage of int16 dtype
df.ItemRank = df.ItemRank.fillna(value=0)
df.ItemRank = pd.to_numeric(df.ItemRank, downcast='integer')

In [136]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3558411 entries, 0 to 3558410
Data columns (total 5 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   AnonID     uint32
 1   Query      object
 2   QueryTime  object
 3   ItemRank   int16 
 4   ClickURL   object
dtypes: int16(1), object(3), uint32(1)
memory usage: 101.8+ MB


In [137]:
df.head()

,AnonID,Query,QueryTime,ItemRank,ClickURL
0,142,rentdirect.com,2006-03-01 07:17:12,0,NaN
1,142,www.prescriptionfortime.com,2006-03-12 12:31:06,0,NaN
2,142,staple.com,2006-03-17 21:19:29,0,NaN
3,142,staple.com,2006-03-17 21:19:45,0,NaN
4,142,www.newyorklawyersite.com,2006-03-18 08:02:58,0,NaN


In [141]:
print(df.ItemRank.min())
print(df.ItemRank.max())

0
500


Ideally I would process the Query or ClickURL columns as well. For now, I will just do ClickURL.

In [145]:
from urllib.parse import urlparse

df.ClickURL = df.ClickURL.map(lambda x: urlparse(x).netloc, na_action='ignore')


In [148]:
df.tail(2) # record 3558409 should just be www.greatseats.com

,AnonID,Query,QueryTime,ItemRank,ClickURL
3558409,24969374,orioles tickets,2006-05-31 12:31:57,2,www.greatseats.com
3558410,24969374,baltimore marinas,2006-05-31 12:43:40,0,NaN


## Query cleaning

### YES implement

Given a raw string, return an array of cleaned strings?

[This article](https://www.analyticsvidhya.com/blog/2021/06/text-preprocessing-in-nlp-with-python-codes/) has a basic overview of what a general strategy could be.

- remove punctuation
- tokenize (split whitespace), regex
- remove stopwords, nltk
- lemmatize, nltk

This should be good enough.

### NO implement

Maybe there's more value to keeping them as raw strings?

Not sure.

## Processing data

Let's not preprocess any queries yet. We can always do that later.

Let's group the data by AnonID. -- I'm not sure this does what I think it does.

In [165]:
df_by_id = df.groupby(by='AnonID')

In [174]:
df_by_id.AnonID.unique()

AnonID
142              [142]
217              [217]
993              [993]
1268            [1268]
1326            [1326]
               ...    
24967641    [24967641]
24967786    [24967786]
24968114    [24968114]
24969251    [24969251]
24969374    [24969374]
Name: AnonID, Length: 65516, dtype: object

I actually want a DF for one user

In [176]:
anon_142_df = df.query('AnonID == 142')

In [178]:
anon_142_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 26
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   AnonID     27 non-null     uint32
 1   Query      27 non-null     object
 2   QueryTime  27 non-null     object
 3   ItemRank   27 non-null     int16 
 4   ClickURL   3 non-null      object
dtypes: int16(1), object(3), uint32(1)
memory usage: 1.0+ KB


In [180]:
anon_142_df.head()

,AnonID,Query,QueryTime,ItemRank,ClickURL
0,142,rentdirect.com,2006-03-01 07:17:12,0,NaN
1,142,www.prescriptionfortime.com,2006-03-12 12:31:06,0,NaN
2,142,staple.com,2006-03-17 21:19:29,0,NaN
3,142,staple.com,2006-03-17 21:19:45,0,NaN
4,142,www.newyorklawyersite.com,2006-03-18 08:02:58,0,NaN


Now we are getting somewehere! There are 65516 users in this dataset -- I just want a sample of them. Need to determine how to narrow down this data. Also to filter "trash" data from possible spambots.

testing GH integration